In [1]:
pip install opencv-python mediapipe

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/165.9 kB ? eta -:--:--
     -------------------------------------- 165.9/165.9 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.8 MB 9.2 MB/s eta 0:00:05
   ---------------------------------------- 0.4/38.8 MB 5.8 MB/s eta 0:00:07
   ---------------------------------------- 0.4/38.8 MB 3.9 MB/s eta 0:00:10
    --------------------------------------- 0.6/38.8 MB 3.1 MB/s eta 0:00:13
    --------------------------------------- 0.8/38.8 MB 3.5 MB/s eta 0:00:11
    ----------------------------


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import mediapipe as mp
import tkinter as tk
from threading import Thread

# Inisialisasi MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Global variable to control the camera loop
running = True

# Fungsi untuk menghitung jumlah jari yang diangkat
def count_fingers(hand_landmarks):
    finger_tips = [hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP],
                   hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP],
                   hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP],
                   hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP],
                   hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]]
    
    finger_dips = [hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP],
                   hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP],
                   hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP],
                   hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP],
                   hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP]]

    fingers_up = [tip.y < dip.y for tip, dip in zip(finger_tips, finger_dips)]

    return sum(fingers_up)

# Fungsi untuk memulai kamera dan deteksi gesture
def start_camera():
    global running
    cap = cv2.VideoCapture(0)

    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while running and cap.isOpened():
            ret, frame = cap.read()

            # Ubah warna gambar menjadi RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Deteksi tangan
            hands_results = hands.process(image)

            # Ubah gambar kembali ke BGR untuk ditampilkan
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Variabel untuk menyimpan jumlah jari yang diangkat
            fingers_count = 0

            if hands_results.multi_hand_landmarks:
                for hand_landmarks in hands_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=2),
                        mp_drawing.DrawingSpec(color=(250,44,250), thickness=2, circle_radius=2)
                    )

                    # Hitung jumlah jari yang diangkat
                    fingers_count = count_fingers(hand_landmarks)
                    
                    # Menampilkan jumlah jari yang diangkat pada layar
                    cv2.putText(image, str(fingers_count), (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 4, (255, 0, 0), 5)

            # Tampilkan hasil
            cv2.imshow('Hand Gesture Tracking', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

# Fungsi untuk menghentikan kamera
def close_camera():
    global running
    running = False

# Setup tkinter window
root = tk.Tk()
root.title("OpenCV Camera Control")

# Button untuk menutup kamera
close_button = tk.Button(root, text="Close Camera", command=close_camera)
close_button.pack()

# Menjalankan kamera di thread terpisah
camera_thread = Thread(target=start_camera)
camera_thread.start()

# Menjalankan aplikasi Tkinter
root.mainloop()

# Pastikan thread kamera selesai sebelum keluar
camera_thread.join()
